<a href="https://colab.research.google.com/github/hassanishere/Data-Analytics/blob/main/Final_Hassan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
from datetime import datetime

# Load the dataset
df = pd.read_csv('train.csv')

# Convert connection date to datetime
df['Connect_Date'] = pd.to_datetime(df['Connect_Date'])

# Calculate the number of days since the connection date until today
current_date = datetime.now()
df['days_since_connection'] = (current_date - df['Connect_Date']).dt.days
df.drop('Connect_Date', axis=1, inplace=True)
f = pd.read_csv('train.csv')

# Perform one-hot encoding on the 'Gender' column
df = pd.get_dummies(df, columns=['Gender'])
df['Gender_M'] = df['Gender_M'].astype(int)
df['Gender_F'] = df['Gender_F'].astype(int)

#Perform one-hot encoding on the 'tariff' column
df = pd.get_dummies(df, columns=['tariff'])
df['tariff_CAT 100'] = df['tariff_CAT 100'].astype(int)
df['tariff_CAT 200'] = df['tariff_CAT 200'].astype(int)
df['tariff_CAT 50'] = df['tariff_CAT 50'].astype(int)
df['tariff_Play 100'] = df['tariff_Play 100'].astype(int)
df['tariff_Play 300'] = df['tariff_Play 300'].astype(int)

#Perform one-hot encoding on the 'Usage_Band' column
df = pd.get_dummies(df, columns=['Usage_Band'])
df['Usage_Band_High'] = df['Usage_Band_High'].astype(int)
df['Usage_Band_Low'] = df['Usage_Band_Low'].astype(int)
df['Usage_Band_Med'] = df['Usage_Band_Med'].astype(int)
df['Usage_Band_MedHigh'] = df['Usage_Band_MedHigh'].astype(int)
df['Usage_Band_MedLow'] = df['Usage_Band_MedLow'].astype(int)

#performing one-hot encoding on the 'Handset column'
df = pd.get_dummies(df, columns=['Handset'])


#Removing the column 'high Dropped calls'
df.drop('high Dropped calls', axis=1, inplace=True)

#Removing the column 'Peak_calls_Sum'
df.drop('Peak_calls_Sum', axis=1, inplace=True)
df.drop('OffPeak_calls_Sum', axis=1, inplace=True)
df.drop('Weekend_calls_Sum', axis=1, inplace=True)



#Removing 'No Usage' column
df.drop('No Usage', axis=1, inplace=True)

#Removing 'id' Column
df.drop('id', axis=1, inplace=True)

#Removing 'Tariff_OK' column
df.drop('Tariff_OK', axis=1, inplace = True)

# Print the DataFrame with the new 'days_since_connection' column
print(df)

#Removing the missing values
df_cleaned = df.dropna()
print(df_cleaned)


#Fitting a gradient boosting algorithm on the data
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score , f1_score
from imblearn.over_sampling import SMOTE



#Loading the test set
#df_test = pd.read_csv('cleanedtest_data.csv')
#X_test = df_test

# Split the data into features and target
X = df_cleaned.drop('target', axis=1)
y = df_cleaned['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=42)

#Performing SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)





from sklearn.ensemble import RandomForestClassifier

# Define a custom scoring function for AUC
def custom_auc(y_true, y_pred):
    return roc_auc_score(y_true, y_pred[:, 1])

# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100,random_state=42)

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV with custom scoring function for AUC
grid_search = GridSearchCV(rf_model, param_grid, scoring=custom_auc, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_resampled, y_resampled)

# Get the best model from GridSearchCV
best_rf_model = grid_search.best_estimator_

# Predict probabilities on the test set
y_probs = best_rf_model.predict_proba(X_test)

# Calculate AUC on the test set
auc = roc_auc_score(y_test, y_probs[:, 1])
print("AUC on test set:", auc)



predictions = pd.DataFrame({'target_probability': y_probs[:, 1], 'average_cost_min': X_test['average cost min']})

# Sort the predictions based on the highest probability of churn predicted
top_20_predictions = predictions.sort_values(by='target_probability', ascending=False).head(20)

# Print top 20 predictions
print(top_20_predictions)





# Sort the predictions based on the feature 'average cost min' and select top 20 predictions
#predictions = pd.DataFrame({'target_probability': y_probs[:, 1], 'average_cost_min': X_test['average cost min']})
#top_20_predictions = predictions.sort_values(by='average_cost_min', ascending=False).head(20)

# Print top 20 predictions
#print(top_20_predictions)


Saving train.csv to train.csv
       Age      L_O_S  Dropped_Calls  Peak_mins_Sum  OffPeak_mins_Sum  \
0     50.0  29.200000            2.0     153.000000        438.600001   
1     25.0  46.533333            1.0     718.800000        164.700000   
2     46.0  38.066667            1.0     322.800000        123.900000   
3     59.0  27.400000            1.0     317.400001        161.699999   
4     25.0  21.433333            1.0     309.600000        637.800000   
...    ...        ...            ...            ...               ...   
5039  16.0  37.433333            2.0    1169.400001        657.900000   
5040  29.0  49.166667            1.0     405.600000        301.200001   
5041  23.0  16.400000            0.0    2382.000000        477.300000   
5042  46.0  31.866667            2.0     112.200000        230.700000   
5043  61.0  23.333333            0.0     403.200000        290.700000   

      Weekend_mins_Sum  International_mins_Sum  Nat_call_cost_Sum    AvePeak  \
0            

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]
  warnings.warn(


AUC on test set: 0.9278373480707234
      target_probability  average_cost_min
3377                0.98          0.263490
3275                0.94          0.155660
866                 0.94          0.726827
1631                0.91          0.185262
543                 0.91          0.148185
4327                0.91          0.166951
4338                0.90          0.148001
409                 0.90          0.193226
4078                0.90          0.174552
1758                0.89          0.184247
4554                0.88          0.348823
3076                0.88          0.129514
586                 0.87          0.169780
2685                0.87          0.137304
2126                0.87          0.244385
2742                0.87          0.185551
2997                0.86          0.140684
4093                0.86          0.159225
2507                0.86          0.163320
4182                0.86          0.149697


In [3]:
print(top_20_predictions)
# Calculate the sum of 'average_cost_min' for the top 20 predictions
total_average_cost_min = top_20_predictions['average_cost_min'].sum()

# Print the sum
print("Sum of average_cost_min for top 20 predictions:", total_average_cost_min)

      target_probability  average_cost_min
3377                0.98          0.263490
3275                0.94          0.155660
866                 0.94          0.726827
1631                0.91          0.185262
543                 0.91          0.148185
4327                0.91          0.166951
4338                0.90          0.148001
409                 0.90          0.193226
4078                0.90          0.174552
1758                0.89          0.184247
4554                0.88          0.348823
3076                0.88          0.129514
586                 0.87          0.169780
2685                0.87          0.137304
2126                0.87          0.244385
2742                0.87          0.185551
2997                0.86          0.140684
4093                0.86          0.159225
2507                0.86          0.163320
4182                0.86          0.149697
Sum of average_cost_min for top 20 predictions: 4.174684


In [46]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
from datetime import datetime

# Load the dataset
df_t = pd.read_csv('test.csv')

# Convert connection date to datetime
df_t['Connect_Date'] = pd.to_datetime(df_t['Connect_Date'])

# Calculate the number of days since the connection date until today
current_date = datetime.now()
df_t['days_since_connection'] = (current_date - df_t['Connect_Date']).dt.days
df_t.drop('Connect_Date', axis=1, inplace=True)
f = pd.read_csv('train.csv')

# Perform one-hot encoding on the 'Gender' column
df_t = pd.get_dummies(df_t, columns=['Gender'])
df_t['Gender_M'] = df_t['Gender_M'].astype(int)
df_t['Gender_F'] = df_t['Gender_F'].astype(int)

#Perform one-hot encoding on the 'tariff' column
df_t = pd.get_dummies(df_t, columns=['tariff'])
df_t['tariff_CAT 100'] = df_t['tariff_CAT 100'].astype(int)
df_t['tariff_CAT 200'] = df_t['tariff_CAT 200'].astype(int)
df_t['tariff_CAT 50'] = df_t['tariff_CAT 50'].astype(int)
df_t['tariff_Play 100'] = df_t['tariff_Play 100'].astype(int)
df_t['tariff_Play 300'] = df_t['tariff_Play 300'].astype(int)

#Perform one-hot encoding on the 'Usage_Band' column
df_t = pd.get_dummies(df_t, columns=['Usage_Band'])
df_t['Usage_Band_High'] = df_t['Usage_Band_High'].astype(int)
df_t['Usage_Band_Low'] = df_t['Usage_Band_Low'].astype(int)
df_t['Usage_Band_Med'] = df_t['Usage_Band_Med'].astype(int)
df_t['Usage_Band_MedHigh'] = df_t['Usage_Band_MedHigh'].astype(int)
df_t['Usage_Band_MedLow'] = df_t['Usage_Band_MedLow'].astype(int)

#performing one-hot encoding on the 'Handset column'
df_t = pd.get_dummies(df_t, columns=['Handset'])


#Removing the column 'high Dropped calls'
df_t.drop('high Dropped calls', axis=1, inplace=True)

#Removing the column 'Peak_calls_Sum'
df_t.drop('Peak_calls_Sum', axis=1, inplace=True)
df_t.drop('OffPeak_calls_Sum', axis=1, inplace=True)
df_t.drop('Weekend_calls_Sum', axis=1, inplace=True)

#Removing 'No Usage' column
df_t.drop('No Usage', axis=1, inplace=True)



#Storing the id column

id = df_t['id']

#Removing 'id' Column
df_t.drop('id', axis=1, inplace=True)

#Removing 'Tariff_OK' column
df_t.drop('Tariff_OK', axis=1, inplace=True)

# Print the DataFrame with the new 'days_since_connection' column
print(df_t)

#Removing the missing values
df_t_cleaned = df_t
print(df_t_cleaned)

df_t_cleaned.to_csv('testt_cleaned.csv', index=False)




Saving test.csv to test (9).csv
       Age      L_O_S  Dropped_Calls  Peak_mins_Sum  OffPeak_mins_Sum  \
0     20.0  55.582251            3.0    1075.667903        189.982227   
1     54.0  32.067010            0.0     378.078657        551.817423   
2     28.0  26.621954            0.0    2064.343139        152.731006   
3     26.0  27.652735            5.0     999.111887        358.760706   
4     34.0  25.479355            0.0     742.647381        141.180280   
...    ...        ...            ...            ...               ...   
1677  32.0  40.097166            0.0    1412.608159         42.952890   
1678  19.0  22.594431            0.0     685.503276         73.643988   
1679  34.0  55.339646            0.0    1945.834456        278.414345   
1680  21.0  42.830741            2.0     419.242872        196.742943   
1681  15.0  48.586523            2.0     182.518726        570.140734   

      Weekend_mins_Sum  International_mins_Sum  Nat_call_cost_Sum    AvePeak  \
0          

In [49]:

df_imputed = df_t_cleaned.fillna(df.median())
print(df_imputed)

import pandas as pd

# Check for missing values and display the rows with missing values
rows_with_missing_values = df_t_cleaned[df_t_cleaned.isnull().any(axis=1)]

# Display rows with missing values
print(rows_with_missing_values)

import pandas as pd

# Check for missing values and display the count of missing values in each column
missing_values = df_t_cleaned.isnull().sum()

# Display columns with missing values
print("Columns with missing values:")
print(missing_values[missing_values > 0])

       Age      L_O_S  Dropped_Calls  Peak_mins_Sum  OffPeak_mins_Sum  \
0     20.0  55.582251            3.0    1075.667903        189.982227   
1     54.0  32.067010            0.0     378.078657        551.817423   
2     28.0  26.621954            0.0    2064.343139        152.731006   
3     26.0  27.652735            5.0     999.111887        358.760706   
4     34.0  25.479355            0.0     742.647381        141.180280   
...    ...        ...            ...            ...               ...   
1677  32.0  40.097166            0.0    1412.608159         42.952890   
1678  19.0  22.594431            0.0     685.503276         73.643988   
1679  34.0  55.339646            0.0    1945.834456        278.414345   
1680  21.0  42.830741            2.0     419.242872        196.742943   
1681  15.0  48.586523            2.0     182.518726        570.140734   

      Weekend_mins_Sum  International_mins_Sum  Nat_call_cost_Sum    AvePeak  \
0             5.017969               25.872

<ipython-input-49-6da31b60fd10>:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_imputed = df_t_cleaned.fillna(df.median())


In [17]:
# Save the DataFrame to a CSV file
df_t_cleaned.to_csv('testt_cleaned.csv', index=False)

# Download the CSV file to your local system
from google.colab import files
files.download('testt_cleaned.csv')




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
import pandas as pd

# Load the cleaned test dataset
#testt_cleaned = pd.read_csv('testt_cleaned.csv')

# Preprocess the test dataset to align with the format of the training dataset
# Make sure to handle any preprocessing steps such as missing values, encoding categorical variables, etc.

# Assuming 'test_cleaned' has been preprocessed and contains the same features as 'X_train'

# Use the trained Random Forest model to make predictions on the test dataset
predictions = best_rf_model.predict(df_imputed)

# You can then use the predictions for further analysis or evaluation
print(predictions)

# Create a DataFrame to show predictions along with other characteristics
predictions_df = pd.DataFrame({'target': predictions})

# Concatenate the original characteristics of the test_cleaned dataset with the predictions
resultt_df = pd.concat([df_t_cleaned, predictions_df], axis=1)

# Concatenate the 'id' column from test_cleaned with the predictions
#resultt_df = pd.concat([df_t_cleaned['id'], predictions_df], axis=1)
resultt_df['id'] = id


# Display the DataFrame
print(resultt_df)

final_df = resultt_df.loc[:, ['id', 'target']]
final_df.rename(columns={'id': 'ID', 'target': 'PRED'}, inplace=True)
final_df.dropna(inplace=True)

print(final_df)

[0 0 0 ... 0 0 0]
       Age      L_O_S  Dropped_Calls  Peak_mins_Sum  OffPeak_mins_Sum  \
0     20.0  55.582251            3.0    1075.667903        189.982227   
1     54.0  32.067010            0.0     378.078657        551.817423   
2     28.0  26.621954            0.0    2064.343139        152.731006   
3     26.0  27.652735            5.0     999.111887        358.760706   
4     34.0  25.479355            0.0     742.647381        141.180280   
...    ...        ...            ...            ...               ...   
1677  32.0  40.097166            0.0    1412.608159         42.952890   
1678  19.0  22.594431            0.0     685.503276         73.643988   
1679  34.0  55.339646            0.0    1945.834456        278.414345   
1680  21.0  42.830741            2.0     419.242872        196.742943   
1681  15.0  48.586523            2.0     182.518726        570.140734   

      Weekend_mins_Sum  International_mins_Sum  Nat_call_cost_Sum    AvePeak  \
0             5.017969   

In [52]:
# Save the DataFrame to a CSV file
final_df.to_csv('Predicted.csv', index=False)

# Download the CSV file to your local system
from google.colab import files
files.download('Predicted.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
# Calculate the percentage of churners and non-churners
churn_percentage = (resultt_df['target'].sum() / len(resultt_df)) * 100
non_churn_percentage = 100 - churn_percentage

# Display the DataFrame
print("Resulting DataFrame:")
print(resultt_df)

# Display the percentages
print("\nPercentage of churners:", churn_percentage)
print("Percentage of non-churners:", non_churn_percentage)

Resulting DataFrame:
       Age      L_O_S  Dropped_Calls  Peak_mins_Sum  OffPeak_mins_Sum  \
0     20.0  55.582251            3.0    1075.667903        189.982227   
1     54.0  32.067010            0.0     378.078657        551.817423   
2     28.0  26.621954            0.0    2064.343139        152.731006   
3     26.0  27.652735            5.0     999.111887        358.760706   
4     34.0  25.479355            0.0     742.647381        141.180280   
...    ...        ...            ...            ...               ...   
1677  32.0  40.097166            0.0    1412.608159         42.952890   
1678  19.0  22.594431            0.0     685.503276         73.643988   
1679  34.0  55.339646            0.0    1945.834456        278.414345   
1680  21.0  42.830741            2.0     419.242872        196.742943   
1681  15.0  48.586523            2.0     182.518726        570.140734   

      Weekend_mins_Sum  International_mins_Sum  Nat_call_cost_Sum    AvePeak  \
0             5.017969